# Anomaly Detection with ERDC Metrics Dataset and Machine Learning Algorithms

## Introduction

The purpose of this assignment is to provide a comprehensive understanding of anomaly detection using machine learning models. We will go through the preprocessing and cleaning the dataset. Later you will then work with the data and create multiple anomaly detection models to analyze the ERDC Metrics data.

It's essential to understand that there are numerous ways to approach anomaly detection, and this guide does not present the only correct method. However, our goal is to equip you with a solid understanding of the thought process and methodology required to achieve accurate results. Different goals and purposes may demand adjustments and alternative approaches.

Goal of the assignment:

In this assignment, our specific goal is to create an anomaly detection system capable of identifying extreme value anomalies that occur over time. To accomplish this, we will test three machine learning estimators: K-Nearest Neighbors (KNN), AutoEncoder and Isolation Forest (iForest). It's important to note that for different purposes, there may be other better estimators to use. Each unique anomaly detection objective requires a carefully selected approach tailored to the specific problem.

Throughout the assignment, we will focus on analyzing data from a different approaches. However, there are some assumptions that will be made before we start. Our servers is based on prior knowledge that the servers operate independently of each other. However the features in each server will remain unknown and could be dependent on different aspects. 

By the end of this assignment, you will have gained valuable insights into the processes involved in anomaly detection using machine learning models. You will learn how to clean and preprocess data, select appropriate estimators, and fine-tune hyperparameters to achieve accurate anomaly detection results. Remember, the journey to an effective anomaly detection system may vary, and it all starts with a well-defined goal and the right methodology. Let's begin our exploration into the world of anomaly detection with machine learning.

## Anomaly

To create a good anomaly detection model, we first need to understand what an anomaly is. The most common anomaly is an extreme value in a dataset, which one datapoint deviate with a higher or lower number than the rest of the datapoints. However, an anomaly could be anything that deviates from "normal" and therefore it is important to know what "normal" is in your case. Another example of anomaly is if the mean of a feature get higher and higher over time until it crashes. In our example we will try to find one case of anomaly and it is the outliers/extreme points that deviates from the clusters or normal instances.

## Background of the dataset and classes

In this section, we will introduce the datasets and tools used in our study. The datasets are imported from the wara ops portal and are divided into three files: "normal", "text" and "string." The "normal" file contains float numbers or integersas values. Each file represents data recorded over a day. The datasets cover multiple servers/hosts, and various features are recorded for each server. We will not work with the text or string files, because we want to find anomalies within extreme values. 

Below is two images of the room with all the servers. The left image shows an overview of a row of servers and to the right we can see one locker with multiple servers. All the servers have a unique name dependent on their location in the room. They start with ese (Ericsson Sweden), then ld (Lund). For the servers we will look at, they have 5 letters or numbers that tells us where it is located.

An example of how a server could be named is eselda06u13. e-Ericsson, se-Sweden, ld-Lund combines to "eseld". This specific server is then located at row a, rack 6 and is found as u13. There are some exceptions of other names, but in this guide we will not focus on them. We will only look at server that are eselda and formated in the same way as the example above.

### The pictures should be in the old git, I don't have access to that git anymore Note: Johan could you find these, otherwise I could remove it

<table><tr>
<td style="text-align:center;"> <h3> A row in the server room</h3> 
    <img src="blinkenlights_many.jpeg" style="width:400px;height:400px;"> </td>
<td style="text-align:center;"> <h3>A locker of racks with servers </h3> 
    <img src="blinkenlights_one.jpeg" style="width:400px;height:400px;"> </td>
</tr></table>



### Toolkits

Pandas:
Pandas is a powerful open-source data analysis and manipulation library. It provides fast and efficient data structures to work with structured data, making it easier to handle and analyze datasets. We will use Pandas to read, explore, and preprocess the imported datasets.

PyOD, NumPy, Pythresh and scikit-learn:
These are additional tools that aid in creating and training machine learning models.

PyOD (Python Outlier Detection) is a library specifically designed for outlier detection algorithms.
NumPy is a fundamental library for numerical computing in Python and provides support for working with arrays and matrices.
Scikit-learn is a comprehensive machine learning library that contains various tools for classification, regression, clustering, and more.
PyThresh is a thresholding outlier detection toolkit and is complementary to work with PyOD. 
Each tool plays a crucial role in facilitating our analysis and model creation process, allowing us to make informed decisions throughout our study. For more in-depth information about each tool, you can refer to their respective websites.

All tools and classes are imported below.

In [ ]:
!pip install pyod
!pip install tensorflow
!pip install pythresh

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import preprocessing, decomposition, model_selection, metrics
from sklearn.pipeline import Pipeline
from sklearn import cluster
from sklearn.manifold import TSNE
from sklearn.linear_model import LinearRegression
from pyod.utils.data import generate_data_clusters
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.auto_encoder import AutoEncoder
from pythresh.thresholds.zscore import ZSCORE
from collections import Counter

## Cleaning the data

To begin the data preprocessing and cleaning process, it is crucial to have a comprehensive understanding of the data we are working with. Drawing meaningful conclusions and making informed assumptions require a solid grasp of the dataset.

In our example, the first step is to read in all the pkl (Python Pickle) files and merge them into one large pandas dataframe.

Data:
- Duration: Ten days
- Purpose: Used for training our machine learning models and find anomalies
- File content: Merged data from multiple pkl files

Our data is unsupervised data and therefore there isn't necessary to divide it into training, test and validation sets. Instead we will use one dataset and try to find anomalies and outliers without knowing the content and if there exists outliers.

In the subsequent sections, we will dive deeper into the data cleaning process, handle missing values, and perform any necessary reshaping to create a well-prepared dataset for training our machine learning models. Remember, understanding the data is the key to successful anomaly detection, and careful preprocessing lays the foundation for accurate results. Let's proceed with the data cleaning and preparation steps. We will in the first step get all files and merge the trainingdata into one pandas DataFrame.

In this example, we will import data from the portal and use the ERDC Metrics dataset. To get the data and see what files are available there is a class called DataportalClient. You can see all possible functions to upload, view and more at "help(DataportalClient)".

In [ ]:
from dataportal import DataportalClient;
#Get a seperate token on portal homepage
token = ''
client = DataportalClient(token)
client.fromDataset('ERDCmetrics').listFiles()

Below is a simple example of how to combine multiple files to get a timeseries longer than a day. We picked to get the first 10 days/files as our data. Above we can see all files and it is important in our case to pick the data with float and integer points and not strings. 

In [ ]:
# Fetch the files
filelist = client.listFiles(prettyPrint=False, returnList=True)

# decide the number of files to be included in the dataset
amount_of_files = 10

df_raw = pd.DataFrame()
for (i, file) in enumerate(filelist):
    if i >= amount_of_files:
        break

    print(f'{i+1}/{amount_of_files} fetching data from {file["MFileName"]}')
    df_raw = pd.concat([df_raw, client.getData(file['FileID'])])

Below we will start to preview and understand our dataset, before we start working on it. First we check all the columns name and you can on you own try to view and understand what each column shows. Our response variable will be "value" and we will try to utilize different variables to find the anomalies in "value" column.

In [ ]:
# Data ordered as records, some preprocessing needed
print('')
df_raw.info(memory_usage='deep')
df_raw.head(5)

Now that we have some kind of understanding of the data, we will start off by checking how much data each feature stores. We will also remove irrelevant columns and create a pivot table of all features (columns) over time (rows).

First we check which features in the name column has most data available. After that we will continue to change the time-column and create our pivot-table with all features available for each server.

In [ ]:
df_raw.name.value_counts()

In [ ]:
df_raw.host.value_counts()

Above we see how much data each server contains. For simplicity we will only work with one server and we will only keep eselda06u17. 

### Task 1 Pick a relevant host and features

Task one is to test which server is best applied for future research. Use e.g. df.loc['Zabbix server'] to view a host and pick based on how many values they contain.

In [ ]:
#Write your solution here


server = '' ##Write your server name here
features = [''] #Write all features in this list

In [ ]:
# Extract the columns corresponding to measurement points, timestamps and values
df_tables = df_raw.loc[:,['host', 'name', 'clock', 'value']]

# # Transform clock from timestamp to datetime, and bin it to minutes
df_tables['clock'] = pd.to_datetime(df_tables['clock'], unit='s').dt.floor('min')

#Keep only eselda06u13 server
df_tables = df_tables[df_tables['host'] == server]

#Keep only features list
df_table = df_tables.loc[df_tables['name'] == features[0]]
for i in range(len(features)-1):
    df_object = df_tables.loc[df_tables['name'] == features[i+1]]
    df_table = pd.concat([df_table, df_object])
    
# Pivot the dataframe to put the measurement points as columns, hosts & timestamps as row indices
# and the rows the values for each measurement point for a timestamp.
df_table = df_table.pivot_table(index=['clock'], columns='name', values='value')

# sort based on host then time
df_table.sort_values(['clock'], inplace=True)

display(df_table)

### Task 2 Clean all NaN

Try to evaluate what to do with each NaN and motivate your decision. Should we remove each row including an NaN, can we set all NaN to zeros? Or is there alternative solutions to this problem? There are three ways to remove NaN, see these methods: [this link for example to remove](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html), [this link for example to interpolate](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html) or replace with an value e.g. 0.

We also see how many NaN exists in your server at the moment below.

In [ ]:
print('nan remaining: ', df_table.isna().to_numpy().sum())

In [ ]:
# Write your solution here. The final dataframe should go into the 'df' variable.


df = df_tables #Keep this

If you have correct replaced or removed all NaN, this code below should print 0

In [ ]:
print('nan remaining: ', df.isna().to_numpy().sum())

## Preprocessing and decomposition

Before building a model, it's essential to preprocess the data. Each machine learning model has its own requirements and limitations, so preprocessing helps make the model faster and avoids potential issues. For now, we will use two techniques: normalization with StandardScaler and dimensionality reduction with PCA. Using StandardScaler, we'll scale the features to have zero mean and unit variance. PCA will reduce the number of dimensions while keeping important information. When using a pipeline, trying different normalizers can be beneficial if the initial choice doesn't work well for the data. This flexibility allows us to fine-tune the preprocessing steps for the best model performance. Let's proceed with data preprocessing before building our anomaly detection model.

### Normalizer

Normalization is a preprocessing technique used in machine learning to to scale features into similar ranges. It aims to bring all features into a closer scale, eliminating the potential bias that may arise from features with larger magnitudes dominating the learning process. 

One of the normalization methods is StandardScaler, which is an implemented preprocessing module in scikit-learn library. StandardScaler follows the principle of standardization, where it transforms the data to be standard normally distributed with zero mean and unit variance. The difference between Normalizer and StandardScaler is the method of scaling. StandardScaler uses scaling of features (columns) of a dataset, while Normalizer scales the rows of a dataset.

StandardScaler is especially useful when the features in the dataset have different scales or variances. It helps to make the optimization process more efficient, particularly for algorithms that rely on distance-based calculations, such as k-nearest neighbors (KNN) and support vector machines (SVM). By bringing the features to a standard scale, StandardScaler prevents features with larger ranges from dominating the learning process and ensures that all features contribute equally. There are multiple other normalizer methods and others could be more effective in other cases. Read more about the others in scikit learns preprocessing API reference. One limitation with StandardScaler is that it is sensitive to outliers and skewed data, which influence the variance and mean in a negative way.

First step is to check if standardization is neccessary. If the mean and variance of the features are close to eachother, standardization isn't usually neccessary. 

In [ ]:
df.mean()

We can see that some cases the mean differ alot and therefore a normalizer would be prefered to make the features contribute equally. Let's also check the variance and distribution before proceeding with a normalizer. If they are similiar to eachother, it means that standardization won't be neccessary.

In [ ]:
df.var()

The dataset exhibits significant differences in variance among its features. To prepare the data for dimensionality reduction, it is crucial to normalize it. However, before proceeding, we need to check if the dataset is strongly skewed.

It's worth mentioning that StandardScaler assumes the data follows a normal distribution or at least is not strongly skewed. If the data violates these assumptions, alternative normalization techniques should be considered to ensure accurate preprocessing.

In [ ]:
X_skew = df.skew().sort_values(ascending=False)
plt.plot(range(1,len(X_skew)+1), X_skew)
plt.xlabel('features')
plt.ylabel('skewness (0 is normal distributed)')
plt.title('Skewness of each feature')
plt.show()

In our dataset, some features show strong skewness, which means their distributions are not symmetrical. Despite this, we will attempt to apply StandardScaler, assuming that the skewed columns will not significantly impact the overall result. However, if the chosen machine learning estimators struggle to make accurate predictions, we might have to consider using an alternative normalization method.

Now, let's proceed with fitting and transforming the StandardScaler on the dataset.

In [ ]:
scaler = preprocessing.StandardScaler()
df = scaler.fit_transform(df)

### PCA

Next step is to apply a decomposition method. We will use the most common method, PCA. 

Principal Component Analysis (PCA) is a widely used technique in the field of statistics and machine learning for data analysis and dimensionality reduction. It is a powerful tool that allows us to extract the most important features or patterns from high-dimensional datasets, while still preserving the essence of the original data.

PCA transforms a set of correlated variables into a new set of uncorrelated variables called pricipal components. The goal of PCA is to reduce the dimensionality of a dataset while retaining as much information as possible. By projecting the data onto a lower-dimensional subspace defined by the principal components, PCA allows us to eliminate redundant or less informative variables. This is particularly useful when dealing with datasets that have a large number of features, as it helps in visualizing and understanding the underlying structure of the data.

PCA has several applications in various fields. It is commonly used for exploratory data analysis, feature extraction, and data visualization. It can also be employed as a preprocessing step for machine learning algorithms, as it can improve the performance and efficiency of models by reducing the dimensionality of the input data.

However, it is important to note that PCA assumes a linear relationship between variables and may not be suitable for datasets with nonlinear structures. In such cases, nonlinear dimensionality reduction techniques like Kernel PCA may be more appropriate. We will start off with see the linear relationship between features and if the features has a correlation between eachother.

#### Task 2 Dimensionality reduction

For the following part check these assumptions and see which of the features have high correlation by calculation cross-variance. 

Task 2a: plot some timeseries of e.g. CPU/memory. Tendensies to extreme values/patterns?

Task 2b: which features have high correlation? Discuss what/how many to keep. Hint: use [this link for example](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)

Task 2c: do a dimensionalty reduction with PCA, rank them according to explained variance (hint: use [this link for example](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Do the number of features with high variance correspond to the number of features you want to keep?

In [ ]:
#Write your solution here:

### Preprocessing Pipeline

You can make the entire process more efficient by using a pipeline in sklearn. A pipeline allows you to apply multiple transformers and estimators in a specific sequence. Its primary purpose is to bring together various steps that can be cross-validated together, including hyperparameter tuning. If you already have a good understanding of the data, a pipeline can be a faster way to create the optimal model.

However, it's crucial to remember to validate the steps mentioned earlier so that you have an idea of what might be causing any issues if the model doesn't perform as expected.

We will show how to implement a pipeline on the preproccessing steps (Normalizer and PCA).

In [ ]:
pipe = Pipeline([('scaler', preprocessing.StandardScaler()), ('pca', decomposition.PCA(svd_solver = 'full'))])

df_pipeline = pipe.fit_transform(df)

This pipeline does the exact same transformation as all preprocessing steps above if the input for each function is corresponding to your previous PCA.

### Clustering

We will also show a short example of another useful decomposition tool, t-SNE. 

T-distributed stochastic neighbor embedding, also called t-SNE, is a statistical tool to visualize high dimensional data. It converts datapoints with similar values into a joint probability. These points get reduced into a three or two dimensional datasets. However t-SNE has a cost function which is not convex. This can give different results for different initalization points. 

T-SNE is a good method to visualize and get a good idea of the amount of clusters. However when training a model or looking for anomalities, it is recommended to use another dimensionality reduction method, for example PCA. 

We are going to show an example of how to use t-SNE with a generated dataset by pyod. We will decide it will have 4 clusters, but in theory we want to make this discovery by ourself. We are also going to compare the t-SNE with the result of PCA. Our generated data will have 100 features and our goal with t-SNE is to find the amount of clusters in our dataset. 

In [ ]:
# Generate a dataset from PyOD with 100 features
X_gen, _, y_gen, _ = generate_data_clusters(n_train=1000, n_clusters=4, n_features=100, contamination=0.1, random_state=123)

print(f'X_gen is a matrix of size: {X_gen.shape}')
print(f'y_gen is an array of size: {y_gen.shape}')

# Reduce dimensions to 2 from 100 using t-SNE and PCA
tsne = TSNE()
X_tsne = tsne.fit_transform(X_gen)

pca2 = decomposition.PCA(n_components=2, svd_solver='full')
X_pca2 = pca2.fit_transform(X_gen)

print(f'\nX_tsne is a matrix of size: {X_tsne.shape}')
print(f'X_pca2 is a matrix of size: {X_pca2.shape}')

# Plot the result
figures, (ax1, ax2) = plt.subplots(1,2)
ax1.scatter(X_tsne[:,0], X_tsne[:,1], c = y_gen)
ax1.set_title('t-SNE')
ax2.scatter(X_pca2[:,0], X_pca2[:,1], c = y_gen)
ax2.set_title('PCA')

Here we can see that there is 4 clusters on the t-SNE and 4 smaller groups, which could be accounted as anomalities. This is a good way to get a perspective of how the dataset is visualized, however when using dimension reduction to train a model, there are better methods. To compare with the pca, the visualization isn't as good and it is harder to see that there are only 4 clusters in the dataset. 

### Task 3 Clustering

Try now to do some clustering solutions with the ERDC Metrics dataset. Do a t-SNE and PCA and compare these solutions (you mau have to use the dataset that was before your dimensionality reduction on this part).

Task 2a: do some PCA/T-SNE, can you spot any obvious clusters?

 Task 2b: do some clustering (Hint: use [this link for example](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html). Plot it with the t-SNE data. What differs between your discovered clusters?

In [ ]:
#Write your solutions here:

## Anomaly Detection models

### Pythresh

In all anomaly detection models we will show examples from the generated data from the clustering section. However, when you do the tasks, you will implement the ERDC Metrics data that we have been working with.

One of the largest problem with PyOD and other libraries contamination methods are the preset of contamination level. The real life dataset is often chaotic and unpredictable and hard to get some useful information before working with it. Therefore tools such as pythresh could be useful to evaluate and test your unlabeled dataset. Pythresh use the decision scores from PyODs methods and with different scores evaluate and find the most extreme values. Below is three examples which will be used on a fixed dataset to try different anomaly detection methods and then evaluate the different scores with PyThresh to get the most extreme datapoints.

### KNN

K-Nearest Neighbors (KNN) is a non-parametric classification algorithm that can also be utilized for anomaly detection tasks. The KNN algorithm, in the context of anomaly detection, works by classifying new instances based on their proximity to existing data points. The KNN algorithm follows a simple principle: "similar instances are likely to belong to the same class." It assumes that instances belonging to the same class will have similar features and attributes. Therefore, if a new instance deviates significantly from its neighboring instances, it is considered an anomaly or outlier.

Training and choosing K:
In the training phase, the KNN algorithm does not build an explicit model but instead stores the training instances as reference points. Each instance in the dataset is represented as a vector in a multi-dimensional feature space, where each feature represents a particular attribute or characteristic. The algorithm calculates the distance between instances based on some distance metric, in our case it is the Euclidean distance.

The parameter "K" represents the number of nearest neighbors for a point considered for classification. Selecting an appropriate value for K is crucial and depends on the characteristics of the dataset. A small K may lead to overfitting, while a large K can cause oversmoothing and hide anomalies. A common rule of thumb is to use k=sqrt(N)/2, when N is amount of datapoints in training set. To find the optimal K, another method is to try every K between 1 and N and derive a plot between error rate and K denothing values. 

When finding the optimal "K" a search-grid could be a useful tool. Another way is to try some different values and take the one which is best. A rule of thumb is to take squareroot of objects as a middle point and try some values both above and below.

Classifying a new instance:
When a new instance (test datapoints) is presented for the KNN algorithm it identifies the K nearest neighbors to that instance. The distance is calculated between the new instance and all the training instances. It then sorts the K nearest neighbors in ascending order and pick K smallest ones.

Decision Rule:
When the new instance have all K nearest neighbors, a decision rule is applied to classify it. The decision rule can be as simple as a majority vote among the K neighbors or using weighted voting, where closer neighbors have a higher influence. When we use KNN for anomaly detection, the decision rule is often based on a predefined threshold from the training data. If the distance between the Kth neighbor exceeds the threshold, the new instance will be classified as an anomaly. However, in our case we create our own threshold, which is the highest score of the training data. Then all our datapoints in the training data will be seen as "normal" instances. This method will be implemented in all anomaly detection models.

Evaluate the Anomalies:
If you use KNN for supervised learning anomaly detection, you can evaluate the performance of the KNN algorithm for anomaly detection by comparing the prediction with the labeled anomalies. Metrics such as precision, recall, F1-score, or the area under the Receiver Operating Characteristic (ROC) curve can be calculated to assess the algorithm's effectiveness. In our case we don't have supervised data and therefore need to examine the datapoints with highest score. 

In [ ]:
clf_name = 'KNN'
clf = KNN(contamination = 1e-7, n_neighbors= 10)
clf.fit(X_pca2)
print("Original threshold:", clf.threshold_)
print("max score:", max(clf.decision_scores_))

thres = ZSCORE()
y_scores = clf.decision_scores_  # raw outlier scores
# binary labels (0: inliers, 1: outliers)
y_pred = thres.eval(y_train_scores)


# evaluate and print the results
print('\nOn Data:')
evaluate_print(clf_name, y_gen, y_train_scores)

# Plot the result
figures, (ax1, ax2) = plt.subplots(1,2)
ax1.scatter(X_pca2[:,0], X_pca2[:,1], c = y_train_scores)
ax1.set_title('Found outliers')
ax2.scatter(X_pca2[:,0], X_pca2[:,1], c = y_gen)
ax2.set_title('True outliers')

Above is a simple example of how to create an anomaly detection model using KNN method and evaluate an unsupervised data. We use PyThresh to get a score, which we can compare to find outliers. 

### iForest

The Isolation Forest (iForest) algorithm is a popular and efficient tree-based method used for anomaly detection. It leverages the concept of isolating anomalies by constructing isolation trees, which are binary trees designed to isolate abnormal instances in the dataset. The iForest algorithm offers a scalable and robust approach to identifying anomalies, particularly in high-dimensional datasets.

Below is a step-by-step explanation with an example at the end of how the iForest algorithm works for anomaly detection:

Training: When training with an iForest algorithm it starts to construct isolation trees. Each tree is built recursively by randomly selecting a feature and a split value to partition the data. The tree stops growing when a criterion is met, either maximum tree depth or minimum number of instances in a leaf node is achieved. It continues the splitting process until all instances are isolated. 

Isolation Trees: An isolation tree consists of internal nodes and leaf nodes. Internal nodes represent splitting conditions based on selected features, while leaf nodes represent partitions of instances. Each instance is assigned a path in the tree from the root to a leaf node. The length of the path corresponds to the number of edges traversed to reach the leaf node.

Anomaly Score Calculation and Decision Rule: The iForest algorithm assigns an anomaly score to each instance based on its path length in the isolation trees. Instances with fewer edges traversed are considiered more likely to be anomalies, since they require fewer partitions to isolate. A threshold is set to classify instances to be anomalous or normal. The instances with anomaly scores above the threshold are considered anomalies. The threshold is determined using various techniques such as statistical methods and domain knowledge. 

Evaluating Anomalies: The iForest algorithm is common used in unlabeled data and therefore you can't always compare with the labeled anomalies. However, metrics such as precision, recall, F1-score or the area under ROC-curve can be used to evaluate the algorithm's effectiveness. 

In [ ]:
clf_name = 'IForest'
clf = IForest(contamination = 1e-7, random_state=123)
clf.fit(df)

print("Original threshold:", clf.threshold_)
print("max score:", max(clf.decision_scores_))

thres = ZSCORE()
y_scores = clf.decision_scores_  # raw outlier scores
# binary labels (0: inliers, 1: outliers)
y_pred = thres.eval(y_train_scores)


# evaluate and print the results
print('\nOn Data:')
evaluate_print(clf_name, y_gen, y_train_scores)

# Plot the result
figures, (ax1, ax2) = plt.subplots(1,2)
ax1.scatter(X_pca2[:,0], X_pca2[:,1], c = y_train_scores)
ax1.set_title('Found outliers')
ax2.scatter(X_pca2[:,0], X_pca2[:,1], c = y_gen)
ax2.set_title('True outliers')

Some important things to remember when using iForest are the advantages and limitations that we will discuss below. 

Advantages with iForest algorithm: As written above, the iForest algorithm is particularly effective in handling high-dimensional datasets, where traditional distance-based methods may suffer from the curse of dimensionality. It is an unsupervised algorithm and does not require labeled data to train an anomaly detection model. The algorithm is scalable and efficient, as the construction of isolation trees can be parallelized and performed in a random and independent manner. iForest can handle datasets with mixed data types, including numerical and categorical features. One important advantage in our case is that iForest can handle skewed data and isn't dependent on being normal distributed data.

Limitations of iForest algorithm: The most important limitation is that the iForest algorithm may struggle with datasets that contain a high proportion of anomalies, as the ability to distinguish between normal and abnormal instances can be diminished. It is sensitive to the choice of the parameters of number of trees and maximum tree depth as it affects its performance. It can be hard to interpreter the anomalies of the iForest algorithm, as it does not provide explicit information about the nature or characteristics of detected anomalies. 

In our case the iForest method didn't improve our model score. We will test one last method and compare all these models.

### AutoEncoder

Autoencoder are unsupervised neural network models that can be utilized for anomaly tasks. They are particularly effective when having data with only normal instances, as it is able to capture underlying structure and patterns in the data. The anomalies, being different from normal instances, have higher reconstruction errors when being compared to normal instances. By leveraging this property, autoencoders can identify anomalies by measuring the discrepancy between the original input and its reconstructed output.

Here is a step-by-step explanation of how autoencoders work for anomaly detection:

Data Preparation: Autoencoders require a dataset that includes both normal and anomalous instances. Having labeled anomalies helps in evaluating the algorithm's performance, while the normal instances are used in training. 

Architecture: Autoencoders consist of two main parts: an encoder and a decoder. The encoder compresses the input data into a lower-dimensional representation, while the decoder aims to reconstruct the original input from this compressed representation (called hidden_neurons in our case). The encoder and decoder are typically implemented as neural network layers, with the number of hidden layers and neurons determined based on the complexity of the dataset.

Training: During training, the autoencoder learns to reconstruct the input data accurately. The objective is to minimize the reconstruction error, which represents the discrepancy between the original input and its reconstructed output. The autoencoder is trained using only normal instances, aiming to capture the normal patterns and structure of the data. Various optimization techniques, such as stochastic gradient descent or Adam optimization, can be employed to train the autoencoder.

Reconstruction and Error Calculation: Once the autoencoder is trained, it can be used to reconstruct both normal and new instances. The reconstruction error is calculated by comparing the original input with its reconstructed output. Different error metrics can be used, such as mean squared error (MSE) or binary cross-entropy. Instances with high reconstruction errors are likely to be anomalies, as they deviate significantly from the learned normal patterns.

Threshold and Decision Rule: A threshold is set to classify instances as normal or anomalous based on their reconstruction errors. Instances with reconstruction errors above the threshold are considered anomalies, while those below the threshold are classified as normal. The threshold can be determined using techniques such as statistical methods, domain knowledge, or by optimizing a specific performance metric.

Evaluation: The performance of the autoencoder for anomaly detection can be evaluated using various metrics, including precision, recall, F1-score, or the area under the Receiver Operating Characteristic (ROC) curve. Comparison with labeled anomalies in the dataset helps assess the effectiveness of the autoencoder in identifying anomalies.

Let's see how all these steps are implemented in the code below:

In [ ]:
clf_name = 'AutoEncoder'
clf = AutoEncoder(hidden_neurons = [2,1,1,2], random_state=123, verbose=0)
clf.fit(df)
print("Original threshold:", clf.threshold_)
print("max score:", max(clf.decision_scores_))

thres = ZSCORE()
y_scores = clf.decision_scores_  # raw outlier scores
# binary labels (0: inliers, 1: outliers)
y_pred = thres.eval(y_train_scores)


# evaluate and print the results
print('\nOn Data:')
evaluate_print(clf_name, y_gen, y_train_scores)

# Plot the result
figures, (ax1, ax2) = plt.subplots(1,2)
ax1.scatter(X_pca2[:,0], X_pca2[:,1], c = y_train_scores)
ax1.set_title('Found outliers')
ax2.scatter(X_pca2[:,0], X_pca2[:,1], c = y_gen)
ax2.set_title('True outliers')

Hopefully if you have cleaned the data and check one servers relevant features, you will find that the models found something strange and maybe some outliers. One thing to notice here is that the model almost seems to be overfit. To avoid overfit you either change parameters or get more data. However, to find out if it really is overfit you could try some other evaluation methods to see how they compare to f1 and confusion matrix.

Advantages of Autoencoders for Anomaly Detection: Autoencoders can capture complex patterns and structures in the data, making them suitable for anomaly detection in various domains. They can handle high-dimensional datasets and learn meaningful representations without explicit feature engineering. Autoencoders are unsupervised models, allowing for the detection of anomalies without the need for labeled training data.

Limitations of Autoencoders for Anomaly Detection: The performance of autoencoders heavily depends on the quality and representativeness of the training data, as well as the complexity of the anomaly patterns. Determine an appropriate threshold for classifying anomalies can be challenging and may require domain knowledge or additional techniques. Autoencoders may struggle to detect anomalies that are significantly different from the learned normal patterns or occur infrequently.

### Task 4 Create an anomaly detection model

Create an anomaly detection model and do hyperparameter tuning to create similar the results above, but with the ERDC Metrics dataset. Use PyODs anomaly detection methods and evaluate with PyThresh and plot as t-SNE mapping.  What do you see? Can you draw some conclusions from your results? (Hints: Do you see that your scores are above the threshold? Plot and visualiz

<b><i> JOHAN R: remove mention of train/test data, also some hints on how to draw conclusions (what features differ in found anomalies, plot some features with the found anomalies as marks etc.)

In [ ]:
# Write your model here: